In [ ]:
import arviz as az
import bambi as bmb
import matplotlib.pyplot as plt
import numpy as np

from matplotlib.patches import Patch

import pandas as pd

# Turn off logging for NUTS sampler for PyMCMC
import logging

logger = logging.getLogger("pymc")
logger.setLevel(logging.ERROR)

## Evaluating and Selecting Models—Bayesian Edition

Last session, we looked at some tools for model selection with frequentist models. In particular, we looked at AIC and BIC as tools to compare models.

For Bayesian models, the 'confidence' of the model is easily seen by the spread of the estimates, however this is different to the model's performance. To assess model performance we recommend the ELPD ('expected log predictive density'). This can be calculated *either* with LOO (leave-one-out) for with WAIC (widely applicable information criterion); we recommend LOO.

To explain the intuition behind ELPD, consider the plot from the first sprint:

![plot](./linear_preds.png)

For observations (blue) that are in areas with a lot of predictions (orange), the *predictive density* is high. For observations by themselves (few predictions) the predictive density is low. By leaving out each observation in turn (LOO) and considering the density in that area, we get a feel for how well the model predicts in general.


## Example 1—Using the wrong model

First we'll look at what happens if we use a model without mixed effects for the data from last sprint.

In [ ]:
rng = np.random.default_rng(seed=42)

In [ ]:
spelling = pd.read_csv(
    "../../datasets/spelling/spelling.tsv", sep="\t", encoding="Windows-1252"
)
spelling

> NOTE: To perform the ELPD comparisons, we need to tell the fitting method to preserve "log-likelihood outcomes", using the slightly weird incantation:

`idata_kwargs={"log_likelihood": True}`

(this is not done by default because it's a little bit slower. An unfortunate choice in our view...)


In [ ]:
model_hierarchical = bmb.Model(
    "error ~ 0 + gender + education + grade + (1|lemma) + (1|subject_ID) ",
    spelling,
    family="bernoulli",
)
idata_hierarchical = model_hierarchical.fit(
    target_accept=0.9,
    random_seed=rng,
    progressbar=False,
    idata_kwargs={"log_likelihood": True},
)
az.summary(idata_hierarchical, var_names=["gender", "education", "grade"])

In [ ]:
model_non_hierarchical = bmb.Model(
    "error ~ 0 + gender + education + grade ",
    spelling,
    family="bernoulli",
)
idata_non_hierarchical = model_non_hierarchical.fit(
    progressbar=False,
    idata_kwargs={"log_likelihood": True},
)

First we use a "forest plot" to compare the "high density interval" for each predictor. This plot tells us how confident the models are in their estimates. 

Here we have added a custom "transform" to graph the results as Odds (which are easier to understand that logistic model parameters), but it is only for clarity. The main thing to notice is that the most likely parameters for the two models are not _so_ different, but the spread changes a lot!

In [ ]:
_, ax = plt.subplots(figsize=(10, 6))

# Create forestplot with ArviZ, only for the mean.
az.plot_forest(
    [
        idata_hierarchical,
        idata_non_hierarchical,
    ],
    var_names=["gender", "education", "grade"],
    combined=True,
    colors=["orange", "mediumorchid"],
    transform=lambda x: np.exp(x),
    linewidth=2.6,
    markersize=8,
    ax=ax,
)

# Create legend
handles = [
    Patch(label="Hierarchical", facecolor="orange"),
    Patch(label="Non-Hierarchical", facecolor="mediumorchid"),
]

ax.set(
    title="94% HDI - Odds Ratio",
)
ax.axvline(1, linestyle="--", color="black", linewidth=0.7)
legend = ax.legend(handles=handles, loc=4, fontsize=14, frameon=True, framealpha=0.8)
plt.tight_layout()
plt.show()

> EXERCISE: How do you interpret the Odds Ratios?

## Confidently Wrong

Note how the Non-Hierarchical model has tighter estimates! This is because from its point of view there are **many more observations** because the students are not grouped. In other words the students for whom there are many records (they were tested on many words) are treated as if they are all different people. This not only makes the model incorrect (because of bias) but also makes it more confident (because it thinks it has more observations).

> IMPORTANT Just because a model has tight estimates does not mean that it is good or correct!

## ELPD Comparison Plot
There is a built in plot to calculate the ELPD, and compare the predictive power of the models using the LOO method as described. This will automatically rank the models. Higher ELPD is better (and the plot arranges them so the models are ranked from top to bottom). The theory is very complicated, but it is similar 'in spirit' to the AIC/BIC measures. In particular, the ELPD also considers the effective number of parameters to automatically penalise models with more parameters.

In [ ]:
comp = az.compare(
    {
        "heirarchical": idata_hierarchical,
        "non-heirarchical": idata_non_hierarchical,
    }
)
az.plot_compare(comp)
plt.show()

## Example 2—Model Comparison

Once again, opinions differ regarding 'exploratory' analysis, but for the sake of argument, here we repeat the model comparison from last session, but in Bayesian style. Because the models can take a while to fit, we compare just the top 10 models from the AIC/BIC comparisons.

In [ ]:
df = pd.read_csv("../../datasets/gries/05-2_reactiontimes.csv", sep="\t")
df["LENGTH"] = df.CASE.str.len()
df["IMAGEABILITY"] = df.IMAGEABILITY.astype(
    pd.api.types.CategoricalDtype(categories=["lo", "hi"], ordered=True)
)
df["FAMILIARITY"] = df.FAMILIARITY.astype(
    pd.api.types.CategoricalDtype(categories=["lo", "med", "hi"], ordered=True)
)
df

In [ ]:
models = [
    "FAMILIARITY + MEANINGFULNESS + LENGTH",
    "MEANINGFULNESS + LENGTH",
    "MEANINGFULNESS*LENGTH",
    "FREQUENCY + FAMILIARITY + MEANINGFULNESS + LENGTH",
    "FAMILIARITY + IMAGEABILITY + MEANINGFULNESS + LENGTH",
    "FREQUENCY + MEANINGFULNESS + LENGTH",
    "IMAGEABILITY + MEANINGFULNESS + LENGTH",
    "FREQUENCY + FAMILIARITY + IMAGEABILITY + LENGTH",
    "FREQUENCY + IMAGEABILITY + MEANINGFULNESS + LENGTH",
    "FAMILIARITY + IMAGEABILITY + LENGTH",
    "FREQUENCY + FAMILIARITY + LENGTH",
]

In [ ]:
# Run each model, store each inference data object in a dict

results = {}
for m in models:
    print(f"Fitting: {m}")
    model = bmb.Model(
        f"RT ~ {m}",
        df.dropna(),
    )
    idata = model.fit(
        progressbar=False,
        idata_kwargs={"log_likelihood": True},
    )
    results[m] = idata

## Comparison Plot

The plots have two components:
- ELPD (with credible interval) is the overal predictive power of the model
- ELPD difference (in grey) is the **difference** from the reference (best) model.

The ELPD difference is probably the most important. Note that the `FAMILIARITY + MEANINGFULNESS + LENGTH` model is probably worse than the reference model, but that the difference line crosses over, so it is not certainly worse. The second-best model by ELPD (`FREQUENCY + MEANINGFULNESS + LENGTH`) is almost certainly worse (but not by much). These results are qualitatively similar to the conclusions we drew last session.

In [ ]:
comp = az.compare(results)
az.plot_compare(comp, figsize=(12, 10))
plt.show()

Finally, we compare the estimates from the models for `FAMILIARITY + MEANINGFULNESS + LENGTH`. Here are the results from the statsmodels linear regression:

<pre>
===============================================================================================
                                  coef    std err          t      P>|t|      [0.025      0.975]
-----------------------------------------------------------------------------------------------
Intercept                     613.6193     69.711      8.802      0.000     473.034     754.204
C(FAMILIARITY, Diff)[D.lo]    -24.3092     17.225     -1.411      0.165     -59.046      10.428
C(FAMILIARITY, Diff)[D.med]   -10.2538     14.383     -0.713      0.480     -39.260      18.753
MEANINGFULNESS                 -0.1355      0.157     -0.863      0.393      -0.452       0.181
LENGTH                         11.1608      3.006      3.713      0.001       5.099      17.222
</pre>

...and here are the results from `bambi`. As you can see, the estimates and the credible intervals (or confidence intervals on the statsmodels side) are very similar.

In [ ]:
az.summary(results["FAMILIARITY + MEANINGFULNESS + LENGTH"])

# In Summary...

When comparing model results you should consider BOTH the uncertainty of the models *and* their predictive accuracy.

```
Version History

Current: v1.0.0

17/11/24: 1.0.0: first draft, BN
```